In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression as lr
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [2]:
df_covid_1 = pd.read_csv("HIST_PAINEL_COVIDBR_2020_Parte2_30mar2022.csv", sep=';', encoding = 'utf8')
df_covid_2 = pd.read_csv("HIST_PAINEL_COVIDBR_2021_Parte1_30mar2022.csv", sep=';', encoding = 'utf8')
df_enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", sep=';', encoding = "cp1252")

frames = [df_covid_1, df_covid_2]

df_covid_nov = pd.concat(frames)

df_covid_nov = df_covid_nov[['regiao', 'estado', 'municipio', 'data', 'populacaoTCU2019','casosAcumulado','casosNovos','obitosAcumulado','obitosNovos']]

df_covid_nov['Casos_per_capita'] = pd.to_numeric(df_covid_nov['casosAcumulado'], downcast="float")
df_covid_nov['Casos_per_capita'] = np.where(df_covid_nov['Casos_per_capita'] >= 1, df_covid_nov['Casos_per_capita'] / df_covid_nov['populacaoTCU2019'], df_covid_nov['Casos_per_capita'])

df_covid_nov['Mortes_per_capita'] = pd.to_numeric(df_covid_nov['obitosAcumulado'], downcast="float")
df_covid_nov['Mortes_per_capita'] = np.where(df_covid_nov['Mortes_per_capita'] >= 1, df_covid_nov['Mortes_per_capita'] / df_covid_nov['populacaoTCU2019'], df_covid_nov['Mortes_per_capita'])

df_covid_nov = df_covid_nov.loc[df_covid_nov['data'] == '2021-01-17']

df_covid_nov.to_csv('df_covid_nov.csv', sep=';', index=False, encoding='utf-8')

df_enem = df_enem[['TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_COR_RACA', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA','TP_ENSINO','CO_MUNICIPIO_ESC', 'SG_UF_ESC' ,'NO_MUNICIPIO_ESC','TP_LOCALIZACAO_ESC',
'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'Q001','Q002','Q003','Q004','Q005','Q006','Q007','Q008','Q009',
'Q010','Q011','Q012','Q013','Q014','Q015','Q016','Q017','Q018','Q019','Q020','Q021','Q022','Q023','Q024','Q025']]

def calc(row):
    if row['TP_PRESENCA_CN'] == 1 & row['TP_PRESENCA_CN'] == 1 & row['TP_PRESENCA_LC'] == 1 & row['TP_PRESENCA_MT'] == 1:
        return 1
    else:
        return 0

df_enem["Presente"] = df_enem.apply(calc, axis = 1)
df_enem.to_csv('df_enem_TESTE.csv', sep=';', index=False, encoding='cp1252')

df_enem = df_enem[df_enem['TP_ST_CONCLUSAO'] == 2] #APENAS Quem vai se formar em 2020

df_enem.to_csv('df_enem.csv', sep=';', index=False, encoding='cp1252')

df_teste = df_enem
df_teste["Total_Alunos"] = df_teste.groupby(['NO_MUNICIPIO_ESC','SG_UF_ESC'])['NO_MUNICIPIO_ESC'].transform('count')

df_teste_2 = df_teste[['NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'Presente', 'Total_Alunos']]
df_teste_2 = pd.concat([df_teste_2[['NO_MUNICIPIO_ESC', 'SG_UF_ESC']], df_teste_2.groupby(['NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'Total_Alunos'])['Presente'].transform('sum')], axis = 1)
df_teste_2["Total_Alunos"] = df_teste_2.groupby(['NO_MUNICIPIO_ESC', 'SG_UF_ESC'])['NO_MUNICIPIO_ESC'].transform('count')
df_teste_2.drop_duplicates(subset = ["NO_MUNICIPIO_ESC", 'SG_UF_ESC'], keep = 'first', inplace = True)

df_teste_2 = df_teste_2.dropna()

df_teste_2.to_csv('df_teste_2.csv', sep=';', index=False, encoding='utf8')

df_teste_final = pd.read_csv("df_teste_2.csv", sep=';', encoding = "utf8")

df_covid_final = pd.read_csv("df_covid_nov.csv", sep=';', encoding = "utf8")

df_merge = pd.merge(df_teste_final, df_covid_final, left_on = ['NO_MUNICIPIO_ESC', 'SG_UF_ESC'], right_on=['municipio','estado'], how='left')

df_merge.drop_duplicates(subset = ['NO_MUNICIPIO_ESC', 'SG_UF_ESC'], keep = 'first', inplace = True)
df_merge.drop_duplicates(subset = ['municipio','estado'], keep = 'first', inplace = True)

df_merge['Relacao_Presenca'] = pd.to_numeric(df_merge['Total_Alunos'], downcast="float")
df_merge['Relacao_Presenca'] = np.where(df_merge['Total_Alunos'] > 0, df_merge['Presente'] / df_merge['Total_Alunos'], df_merge['Total_Alunos'])

df_merge.to_csv('df_merge.csv', sep=';', index=False, encoding='utf8')

In [3]:
frames = [df_covid_1, df_covid_2]

df_covid_nov = pd.concat(frames)

df_covid_nov = df_covid_nov[['regiao', 'estado', 'municipio', 'data', 'populacaoTCU2019','casosAcumulado','casosNovos','obitosAcumulado','obitosNovos']]

datas_antigas = ["2021-11-21","2021-11-20","2021-11-19","2021-11-18","2021-11-17","2021-11-16","2021-11-15","2021-11-14","2021-11-13","2021-11-12","2021-11-11","2021-11-10",
"2021-11-09","2021-11-08","2021-11-07","2021-11-06","2021-11-05","2021-11-04","2021-11-03","2021-11-02","2021-11-01","2021-10-31","2021-10-30","2021-10-29","2021-10-28",
"2021-10-27","2021-10-26","2021-10-25","2021-10-24","2021-10-23","2021-10-22"]

datas = ["2021-01-17","2021-01-16","2021-01-15","2021-01-14","2021-01-13","2021-01-12","2021-01-11","2021-01-10","2021-01-09","2021-01-08","2021-01-07","2021-01-06",
"2021-01-05","2021-01-04","2021-01-03","2021-01-02","2021-01-01","2021-01-17","2021-01-17","2021-01-17","2021-01-17","2020-12-31","2020-12-30","2020-12-29","2020-12-28",
"2020-12-27","2020-12-26","2020-12-25","2020-12-24","2020-12-23","2020-12-22","2020-12-21","2020-12-20","2020-12-19","2020-12-18","2020-12-17"]

df_covid_nov = df_covid_nov.loc[df_covid_nov['data'].isin(datas)]

df_covid_nov['Casos_per_capita'] = pd.to_numeric(df_covid_nov['casosAcumulado'], downcast="float")
df_covid_nov['Casos_per_capita'] = np.where(df_covid_nov['Casos_per_capita'] >= 1, df_covid_nov['Casos_per_capita'] / df_covid_nov['populacaoTCU2019'], df_covid_nov['Casos_per_capita'])

df_covid_nov['Mortes_per_capita'] = pd.to_numeric(df_covid_nov['obitosAcumulado'], downcast="float")
df_covid_nov['Mortes_per_capita'] = np.where(df_covid_nov['Mortes_per_capita'] >= 1, df_covid_nov['Mortes_per_capita'] / df_covid_nov['populacaoTCU2019'], df_covid_nov['Mortes_per_capita'])

df_covid_nov.to_csv('df_covid_nov_2_mes_municipio.csv', sep=';', index=False, encoding='utf-8')

df_teste_final = pd.read_csv("df_teste_2.csv", sep=';', encoding = "utf8")

df_covid_final = pd.read_csv("df_covid_nov_2_mes_municipio.csv", sep=';', encoding = "utf8")

df_merge = pd.merge(df_teste_final, df_covid_final, left_on = ['NO_MUNICIPIO_ESC', 'SG_UF_ESC'], right_on= ['municipio', 'estado'], how='left')

df_merge.to_csv('df_merge_2_mes_municipio.csv', sep=';', index=False, encoding='utf8')

df_merge = df_merge.groupby(['NO_MUNICIPIO_ESC', 'SG_UF_ESC'])[['casosNovos','obitosNovos']].agg('sum').reset_index()

df_merge.drop_duplicates(subset = ['NO_MUNICIPIO_ESC', 'SG_UF_ESC'], keep = 'first', inplace = True)

df_merge = df_merge[list(df_merge.columns[~df_merge.columns.duplicated()])]

df_merge.to_csv('df_merge_2_mes_municipio_teste.csv', sep=';', index=False, encoding='utf8')

df_merge_1 = pd.read_csv("df_merge.csv", sep=';', encoding = "utf8")
df_merge_2 = df_merge[['NO_MUNICIPIO_ESC', 'SG_UF_ESC','casosNovos','obitosNovos']]
df_merge_2 = df_merge_2.rename(columns={"casosNovos": "casosNovos_Semana", "obitosNovos": "obitosNovos_Semana"})
df_merge_2['casosNovos_Semana'] = df_merge_2['casosNovos_Semana'].apply(abs)
df_merge_2['obitosNovos_Semana'] = df_merge_2['obitosNovos_Semana'].apply(abs)

df_final = pd.merge(df_merge_1, df_merge_2, left_on = ['NO_MUNICIPIO_ESC', 'SG_UF_ESC'], right_on = ['NO_MUNICIPIO_ESC', 'SG_UF_ESC'], how='left')

df_final = df_final[['municipio','estado','regiao','Presente','Total_Alunos','populacaoTCU2019','casosAcumulado','obitosAcumulado',
'casosNovos_Semana','obitosNovos_Semana','Casos_per_capita','Mortes_per_capita','Relacao_Presenca']]

df_final['Relacao_Presenca'] = pd.to_numeric(df_final['Total_Alunos'], downcast="float")
df_final['Relacao_Presenca'] = np.where(df_final['Total_Alunos'] > 0, df_final['Presente'] / df_final['Total_Alunos'], df_final['Total_Alunos'])

df_final['Casos_per_capita'] = pd.to_numeric(df_final['casosAcumulado'], downcast="float")
df_final['Casos_per_capita'] = np.where(df_final['Casos_per_capita'] >= 1, df_final['Casos_per_capita'] / df_final['populacaoTCU2019'], df_final['Casos_per_capita'])

df_final['Mortes_per_capita'] = pd.to_numeric(df_final['obitosAcumulado'], downcast="float")
df_final['Mortes_per_capita'] = np.where(df_final['Mortes_per_capita'] >= 1, df_final['Mortes_per_capita'] / df_final['populacaoTCU2019'], df_final['Mortes_per_capita'])

df_final['Relacao_casos_mes'] = np.where(df_final['Total_Alunos'] >= 0, df_final['casosNovos_Semana'] / df_final['populacaoTCU2019'], df_final['populacaoTCU2019'])
df_final['Relacao_obitos_mes'] = np.where(df_final['Total_Alunos'] >= 0, df_final['obitosNovos_Semana'] / df_final['populacaoTCU2019'], df_final['populacaoTCU2019'])

df_final.loc[df_final['estado'] == 'AC', 'estado'] = "Acre"
df_final.loc[df_final['estado'] == 'AL', 'estado'] = "Alagoas"
df_final.loc[df_final['estado'] == 'AP', 'estado'] = "Amapá"
df_final.loc[df_final['estado'] == 'AM', 'estado'] = "Amazonas"
df_final.loc[df_final['estado'] == 'BA', 'estado'] = "Bahia"
df_final.loc[df_final['estado'] == 'CE', 'estado'] = "Ceará"
df_final.loc[df_final['estado'] == 'ES', 'estado'] = "Espírito Santo"
df_final.loc[df_final['estado'] == 'GO', 'estado'] = "Goiás"
df_final.loc[df_final['estado'] == 'MA', 'estado'] = "Maranhão"
df_final.loc[df_final['estado'] == 'MT', 'estado'] = "Mato Grosso"
df_final.loc[df_final['estado'] == 'MS', 'estado'] = "Mato Grosso do Sul"
df_final.loc[df_final['estado'] == 'MG', 'estado'] = "Minas Gerais"
df_final.loc[df_final['estado'] == 'PA', 'estado'] = "Pará"
df_final.loc[df_final['estado'] == 'PB', 'estado'] = "Paraíba"
df_final.loc[df_final['estado'] == 'PR', 'estado'] = "Paraná"
df_final.loc[df_final['estado'] == 'PE', 'estado'] = "Pernambuco"
df_final.loc[df_final['estado'] == 'PI', 'estado'] = "Piauí"
df_final.loc[df_final['estado'] == 'RJ', 'estado'] = "Rio de Janeiro"
df_final.loc[df_final['estado'] == 'RN', 'estado'] = "Rio Grande do Norte"
df_final.loc[df_final['estado'] == 'RS', 'estado'] = "Rio Grande do Sul"
df_final.loc[df_final['estado'] == 'RO', 'estado'] = "Rondônia"
df_final.loc[df_final['estado'] == 'RR', 'estado'] = "Roraima"
df_final.loc[df_final['estado'] == 'SC', 'estado'] = "Santa Catarina"
df_final.loc[df_final['estado'] == 'SP', 'estado'] = "São Paulo"
df_final.loc[df_final['estado'] == 'SE', 'estado'] = "Sergipe"
df_final.loc[df_final['estado'] == 'TO', 'estado'] = "Tocantins"
df_final.loc[df_final['estado'] == 'DF', 'estado'] = "Distrito Federal"

df_codigos = pd.read_csv("codigos.csv", sep=',', encoding = "utf8")
df_codigos = df_codigos[['estado','municipio','codigo']]
df_final = pd.merge(df_final, df_codigos, left_on = ['municipio','estado'], right_on= ['municipio','estado'], how='left')

df_final.to_csv('df_covid_final_mes_municipio_dif.csv', sep=';', index=False, encoding='utf8')

df_diferenca = pd.read_csv("taxas_abstencao_municipio.csv", sep=',', encoding = "utf8")###################
df_diferenca = df_diferenca[['codigo','diferenca']]###################
df_final = pd.merge(df_final, df_diferenca, left_on = ['codigo'], right_on= ['codigo'], how='inner')###################

df_final.drop_duplicates(subset = ['municipio', 'estado'], keep = 'first', inplace = True)###################

df_final = df_final.dropna()

df_final.to_csv('df_TESTE.csv', sep=';', index=False, encoding='utf8')

In [4]:
df_escolar_1 = pd.read_csv("dados_escolares_2019.csv", sep=',', encoding = 'cp1252')
df_escolar_2 = pd.read_csv("dados_escolares_2020.csv", sep=',', encoding = 'cp1252')
df_projeto_final = pd.read_csv("df_TESTE.csv", sep=';', encoding = 'utf-8')

df_escolar_1 = df_escolar_1[['Ano','UF','Código do Município','Nome do Município','Localização','Dependência Administrativa','Aprovacao_3_ano','Reprovacao_3_ano','Desistencia_3_ano']]
df_escolar_2 = df_escolar_2[['Ano','UF','Código do Município','Nome do Município','Localização','Dependência Administrativa','Aprovacao_3_ano_2020','Reprovacao_3_ano_2020','Desistencia_3_ano_2020']]

df_projeto_final_arquivo = df_projeto_final[['municipio', 'estado', 'codigo', 'Total_Alunos', 'populacaoTCU2019', 'Casos_per_capita', 'Mortes_per_capita','Relacao_casos_mes','Relacao_obitos_mes','diferenca']]

df_escolar_1 = df_escolar_1.loc[(df_escolar_1['Localização'] == 'Total') & (df_escolar_1['Dependência Administrativa'] == 'Total')]
df_escolar_2 = df_escolar_2.loc[(df_escolar_2['Localização'] == 'Total') & (df_escolar_2['Dependência Administrativa'] == 'Total')]

df_escolar_1 = df_escolar_1[['Código do Município','Aprovacao_3_ano','Reprovacao_3_ano','Desistencia_3_ano']]
df_projeto_final_merge_1 = pd.merge(df_projeto_final_arquivo, df_escolar_1, left_on = ['codigo'], right_on= ['Código do Município'], how='left')

df_escolar_2 = df_escolar_2[['Código do Município','Aprovacao_3_ano_2020','Reprovacao_3_ano_2020','Desistencia_3_ano_2020']]
df_projeto_final_merge_2 = pd.merge(df_projeto_final_arquivo, df_escolar_2, left_on = ['codigo'], right_on= ['Código do Município'], how='left')

df_projeto_final_merge_2 = df_projeto_final_merge_2[['Código do Município','Aprovacao_3_ano_2020','Reprovacao_3_ano_2020','Desistencia_3_ano_2020']]
df_projeto_final = pd.merge(df_projeto_final_merge_1, df_projeto_final_merge_2, left_on = ['Código do Município'], right_on= ['Código do Município'], how='left')

df_projeto_final = df_projeto_final.replace("--", np.nan)

df_projeto_final = df_projeto_final.dropna()

df_projeto_final['Aprovacao_3_ano'] = df_projeto_final['Aprovacao_3_ano'].str.replace(',', '.').astype(float)
df_projeto_final['Reprovacao_3_ano'] = df_projeto_final['Reprovacao_3_ano'].str.replace(',', '.').astype(float)
df_projeto_final['Desistencia_3_ano'] = df_projeto_final['Desistencia_3_ano'].str.replace(',', '.').astype(float)
df_projeto_final['Aprovacao_3_ano_2020'] = df_projeto_final['Aprovacao_3_ano_2020'].str.replace(',', '.').astype(float)
df_projeto_final['Reprovacao_3_ano_2020'] = df_projeto_final['Reprovacao_3_ano_2020'].str.replace(',', '.').astype(float)
df_projeto_final['Desistencia_3_ano_2020'] = df_projeto_final['Desistencia_3_ano_2020'].str.replace(',', '.').astype(float)

df_projeto_final['Diferenca_aprovacao'] = df_projeto_final['Aprovacao_3_ano_2020'] - df_projeto_final['Aprovacao_3_ano']
df_projeto_final['Diferenca_reprovacao'] = df_projeto_final['Reprovacao_3_ano_2020'] - df_projeto_final['Reprovacao_3_ano']
df_projeto_final['Diferenca_desistencia'] = df_projeto_final['Desistencia_3_ano_2020'] - df_projeto_final['Desistencia_3_ano']

df_projeto_final['Relacao_cadastrados'] = df_projeto_final['Total_Alunos'] / df_projeto_final['populacaoTCU2019']

df_projeto_final = df_projeto_final[['municipio', 'estado', 'codigo','Total_Alunos', 'Relacao_cadastrados','Casos_per_capita', 'Mortes_per_capita','Relacao_casos_mes','Relacao_obitos_mes','Diferenca_aprovacao','Diferenca_reprovacao','Diferenca_desistencia','diferenca']]
#Apenas mais que 5 alunos totais.
df_projeto_final = df_projeto_final[(df_projeto_final['Total_Alunos'] >= 5)]

df_projeto_final.to_csv('df_projeto_final_covid_e_escola.csv', sep=';', index=False, encoding='utf-8')